In [ ]:
from skimage.transform import resize
from nilearn import plotting
import nibabel as nib
import numpy as np
import torch
import ipdb
import csv
import os
import re


arr1 = [4,6,8,10,12,14,16,18,20,22,24,26,28,30,32]
arr2 = [34,36,38,40,42,44,46,48,50,52,54,56,58,60]
# show a nifti image
path = '../ADNI/021_S_1109/FreeSurfer_Cross-Sectional_Processing_brainmask/2006-12-05_14_24_09.0/S23446/mri/image.nii'
featmask = nib.load(path)  
disp = plotting.plot_img(featmask,cut_coords=arr1,draw_cross=False,annotate=False,black_bg=True,display_mode='z')
# disp.annotate(size=25,left_right=False,positions=True)
plotting.show()
disp=plotting.plot_img(featmask,cut_coords=arr2,draw_cross=False,annotate=False,black_bg=True,display_mode='z')
# disp.annotate(size=25,left_right=False)
plotting.show()

In [ ]:
path = '../ADNI/021_S_1109/FreeSurfer_Cross-Sectional_Processing_brainmask/2006-12-05_14_24_09.0/S23446/mri/image.nii'
mgz_path = '../ADNI/021_S_1109/FreeSurfer_Cross-Sectional_Processing_brainmask/2006-12-05_14_24_09.0/S23446/mri/brainmask.mgz'
!mri_convert ../ADNI/021_S_1109/FreeSurfer_Cross-Sectional_Processing_brainmask/2006-12-05_14_24_09.0/S23446/mri/brainmask.mgz ../ADNI/021_S_1109/FreeSurfer_Cross-Sectional_Processing_brainmask/2006-12-05_14_24_09.0/S23446/mri/image_save.nii

In [ ]:
!mri_convert ../ADNI/021_S_1109/FreeSurfer_Cross-Sectional_Processing_brainmask/2006-12-05_14_24_09.0/S23446/mri/brainmask.mgz ../ADNI/021_S_1109/FreeSurfer_Cross-Sectional_Processing_brainmask/2006-12-05_14_24_09.0/S23446/mri/image.nii
path = '../ADNI/021_S_1109/FreeSurfer_Cross-Sectional_Processing_brainmask/2006-12-05_14_24_09.0/S23446/mri/image.nii'
mgz_path = '../ADNI/021_S_1109/FreeSurfer_Cross-Sectional_Processing_brainmask/2006-12-05_14_24_09.0/S23446/mri/brainmask.mgz'
padding = 5
arr = range(0, 255, 10)
# show mgz 
print(featmask.shape)
featmask = nib.load(mgz_path)  
plotting.plot_img(featmask,cut_coords=arr,draw_cross=False,annotate=False,black_bg=True,display_mode='x')
plotting.plot_img(featmask,cut_coords=arr,draw_cross=False,annotate=False,black_bg=True,display_mode='y')
plotting.plot_img(featmask,cut_coords=arr,draw_cross=False,annotate=False,black_bg=True,display_mode='z')

img = nib.load(path)
np_img = np.array(img.get_fdata())
    
# clean X 
x_len, y_len, z_len = len(np_img), len(np_img[0]), len(np_img[0, 0])
zero_mask = np.zeros((y_len, z_len)) 
del_inds = []
for i in range(x_len):
    if np.sum(np_img[i] != zero_mask) == 0:
        del_inds.append(i)
np_img = np.delete(np_img, del_inds, 0)
# add paddings
np_img = np.insert(np_img, [0 for i in range(padding)], zero_mask, 0)
np_img = np.insert(np_img, [len(np_img) for i in range(padding)], zero_mask, 0)

# clean Y
x_len, y_len, z_len = len(np_img), len(np_img[0]), len(np_img[0, 0])
zero_mask = np.zeros((x_len, z_len)) 
del_inds = []
for i in range(y_len):
    if np.sum(np_img[:, i] != zero_mask) == 0:
        del_inds.append(i)
np_img = np.delete(np.swapaxes(np_img, 0, 1), del_inds, 0)
np_img = np.swapaxes(np_img, 0, 1)
# add paddings
np_img = np.insert(np_img, [0 for i in range(padding)], np.zeros(z_len), 1)
np_img = np.insert(np_img, [len(np_img[0]) for i in range(padding)], np.zeros(z_len), 1)

# clean Z
x_len, y_len, z_len = len(np_img), len(np_img[0]), len(np_img[0, 0])
zero_mask = np.zeros((x_len, y_len)) 
del_inds = []
for i in range(z_len):
    if np.sum(np_img[:, :, i] != zero_mask) == 0:
        del_inds.append(i)
np_img = np.delete(np.swapaxes(np_img, 0, 2), del_inds, 0)
np_img = np.swapaxes(np_img, 0, 2)
# add paddings
np_img = np.insert(np_img, [0 for i in range(padding)], 0, 2)
np_img = np.insert(np_img, [len(np_img[0][0]) for i in range(padding)], 0, 2)

# write to disk
feat = nib.Nifti1Image(np_img, img.affine)
nib.save(feat, path)

# show image
img = nib.load(path)
plotting.plot_img(feat,cut_coords=arr,draw_cross=False,annotate=False,black_bg=True,display_mode='x')
plotting.plot_img(feat,cut_coords=arr,draw_cross=False,annotate=False,black_bg=True,display_mode='y')
plotting.plot_img(feat,cut_coords=arr,draw_cross=False,annotate=False,black_bg=True,display_mode='z')
plotting.show()


In [ ]:
path = '../ADNI/023_S_0331/FreeSurfer_Cross-Sectional_Processing_brainmask/2010-05-05_10_12_35.0/S84476/mri/brainmask.mgz'
arr = range(0, 64, 4)
nii = nib.load(path)

img = np.swapaxes(nii.get_fdata(),1,2)
img = np.flip(img,1)
img = np.flip(img,2)
sp_size = 64
img = resize(img, (sp_size,sp_size,sp_size), mode='constant')
img = np.maximum(0, img * 2 - 1)
imageout = torch.from_numpy(img).float().view(1,sp_size,sp_size,sp_size)
feat = nib.Nifti1Image(img, np.eye(4))
# show image
print(img.shape)
plotting.plot_img(feat,cut_coords=arr,draw_cross=False,annotate=False,black_bg=True,display_mode='x')
plotting.plot_img(feat,cut_coords=arr,draw_cross=False,annotate=False,black_bg=True,display_mode='y')
plotting.plot_img(feat,cut_coords=arr,draw_cross=False,annotate=False,black_bg=True,display_mode='z')
plotting.show()


path = '../ADNI/023_S_0331/FreeSurfer_Cross-Sectional_Processing_brainmask/2010-05-05_10_12_35.0/S84476/mri/image.nii'
arr = range(0, 64, 4)
nii = nib.load(path)

img = np.swapaxes(nii.get_fdata(),1,2)
img = np.flip(img,1)
img = np.flip(img,2)
sp_size = 64
img = resize(img, (sp_size,sp_size,sp_size), mode='constant')
img = np.maximum(0, img * 2 - 1)
imageout = torch.from_numpy(img).float().view(1,sp_size,sp_size,sp_size)
feat = nib.Nifti1Image(img, np.eye(4))
# show image
print(img.shape)
plotting.plot_img(feat,cut_coords=arr,draw_cross=False,annotate=False,black_bg=True,display_mode='x')
plotting.plot_img(feat,cut_coords=arr,draw_cross=False,annotate=False,black_bg=True,display_mode='y')
plotting.plot_img(feat,cut_coords=arr,draw_cross=False,annotate=False,black_bg=True,display_mode='z')
plotting.show()

In [ ]:
with open('../ADNI/clipped.csv', 'w+') as csvfile:
    fwriter = csv.writer(csvfile)
    fnames = os.listdir('../ADNI')
    r = re.compile('^[0-9][0-9][12456789].*')
    fwriter.writerow(list(filter(r.match, fnames)))
    

In [ ]:
np.eye(4)

In [ ]:
!ls ../ADNI/